# Тестовое задание

Требуется разработать алгоритм максимизации функции k на ориентированном мультиграфе (ССиТ).

Построение алгоритма будет выполняться в 2 этапа:
1. Написание движка для нахождения функции k на заданном ориентированном мультиграфе (ССиТ)
2. Оптимизация функции k

### Этап 1: Написание движка для нахождения функции k на заданном ориентированном мультиграфе

In [57]:
from sympy import symbols, Eq, solve

# Создание классов вершин и ребр графа. Скважины и сток будут особыми узлами с отдельными классами
class Well:
    def __init__(self, idx, alpha, const, Q):
        self.idx = idx
        self.alpha = alpha
        self.const = const
        # self.edge_input = edge_input
        self.Q = Q
    # def flow_rate(self):
    #     return self.alpha * (  ) * self.const
          
class Vertex:
    def __init__(self, idx):
        self.idx = idx
        self.edge_input = []
        # self.edge_input = edge_input # List of pipes that are included in the node
        self.Q = 0

    def update_Q(self):
        self.Q = sum(edge.Q_edge for edge in self.edge_input)

class Edge:
    def __init__(self, idx, vertex1, vertex2, length = 5, diameter = 2, roughness = 0.1, density = 1000):
        self.idx = idx
        self.vertex1 = vertex1
        self.vertex2 = vertex2
        self.length = length
        self.diameter = diameter
        self.roughness = roughness
        self.density = density
        self.update_Q_edge()

    def update_Q_edge(self):
        self.Q_edge = self.vertex1.Q
        self.pressure_loss = self.density * self.length * self.Q_edge * self.roughness / self.diameter

class Drain:
    def __init__(self, p_0, alpha = 3, const = 10, idx=0):
        self.p_0 = p_0
        self.edge_input = []
        self.alpha = alpha
        self.const = const
        self.idx = idx
        self.Q_0 = self.alpha * self.p_0 + self.const

In [58]:
# Создание графа
def create_graph():
    alpha = 3
    const = 5
    # get the number of Wells
    num_wells = int(input("Enter the number of Wells: "))
    wells = []
    
    # Создание списка из num_wells элементов - Q1, Q2, Q3
    flow_rate = []
    for i in range(num_wells):
        flow_rate.append('Q' + str(i+1))
    flow_rate = symbols(flow_rate)
    
    counter = 0
    for i in range(num_wells):
        counter = counter + 1
        wells.append(Well(i+1, alpha, const, flow_rate[i]))

    # get the number of Vertex
    num_vertex = int(input("Enter the number of Vertex: "))
    vertex = []
    for i in range(num_vertex):
        vertex.append(Vertex(i + counter))

    # Create Drain
    drain = [Drain(100)]
    common_vertex = drain + wells + vertex # A common array of all vertices
    print(common_vertex)

    # get the number of Edges
    num_edges = int(input("Enter the number of Edges: "))
    edges = []
    for i in range(num_edges):
        vertex1 = int(input("Enter the first vertex for edge {}: ".format(i + 1)))
        vertex2 = int(input("Enter the second vertex for edge {}: ".format(i + 1)))
        # length = float(input("Enter the length for edge {}: ".format(i + 1)))
        # diameter = float(input("Enter the diameter for edge {}: ".format(i + 1)))
        # roughness = float(input("Enter the roughness for edge {}: ".format(i + 1)))
        # edges.append(Edge(i+1, common_vertex[vertex1], common_vertex[vertex2], length, diameter, roughness))
        edges.append(Edge(i+1, common_vertex[vertex1], common_vertex[vertex2]))

    for edge in edges:
        # edge.vertex1.edge_input.append(edge)
        edge.vertex2.edge_input.append(edge)
        
    # Обновление характеристик дебитов на узлах и гранях, после создания всех элементов графа и их связей
    while edges[num_edges-1].Q_edge == 0:
        # for well in wells:
        #     well.flow_rate()

        for edge in edges:
            edge.update_Q_edge()

        for vert in vertex:
            vert.update_Q()

    return wells, vertex, drain, edges, flow_rate

In [59]:
wells, vertex, drain, edges, flow_rate = create_graph()

[<__main__.Drain object at 0x0000017ECF5CD970>, <__main__.Well object at 0x0000017ECF5B0CA0>, <__main__.Well object at 0x0000017ECF5CD2B0>, <__main__.Vertex object at 0x0000017ECF5CD1C0>, <__main__.Vertex object at 0x0000017ECF5CD2E0>, <__main__.Vertex object at 0x0000017ECF5CD3A0>]


In [60]:
# print(edges[0].Q_edge)
# print(edges[1].Q_edge)
# print(edges[2].Q_edge)
print(edges[3].Q_edge)
print(edges[4].Q_edge)
# print(edges[7].Q_edge)
# print(edges[7].pressure_loss)
# print(type(edges[0].Q_edge))
print(wells)

Q2
Q1 + Q2
[<__main__.Well object at 0x0000017ECF5B0CA0>, <__main__.Well object at 0x0000017ECF5CD2B0>]


In [61]:

def find_pipes(well, edges, drain):
    pipes = []
    visited = set()
    queue = []
    queue.append(well)
    while queue:
        curr = queue.pop(0)
        visited.add(curr)
        for edge in edges:
            if edge.vertex1 == curr and edge.vertex2 not in visited:
                queue.append(edge.vertex2)
                pipes.append(edge)
                if edge.vertex2 == drain:
                    return pipes
    return pipes

# pipes_from_well_to_drain = []
# for well in wells:
#     pipes_from_wells_to_drain.append(find_pipes(well, edges, drain[0]))
pipes_from_well_to_drain = find_pipes(wells[1], edges, drain[0])

In [62]:
print(pipes_from_well_to_drain)
print(pipes_from_well_to_drain[0].Q_edge)
print(pipes_from_well_to_drain[1].Q_edge)
print(pipes_from_well_to_drain[2].Q_edge)

[<__main__.Edge object at 0x0000017ECF5CD700>, <__main__.Edge object at 0x0000017ECF5CD490>, <__main__.Edge object at 0x0000017ECF5CD040>]
Q2
Q2
Q1 + Q2


In [63]:
pipes_from_wells_to_drain = []
for well in wells:
    pipes_from_wells_to_drain.append(find_pipes(well, edges, drain[0]))

In [64]:
print(pipes_from_wells_to_drain[1][2].Q_edge)
print(pipes_from_wells_to_drain[0])

Q1 + Q2
[<__main__.Edge object at 0x0000017ECF5CD340>, <__main__.Edge object at 0x0000017ECF5CD280>, <__main__.Edge object at 0x0000017ECF5CD040>]


In [65]:
# print(len(wells))
for well in range(len(wells)):
    print(pipes_from_wells_to_drain[well])

[<__main__.Edge object at 0x0000017ECF5CD340>, <__main__.Edge object at 0x0000017ECF5CD280>, <__main__.Edge object at 0x0000017ECF5CD040>]
[<__main__.Edge object at 0x0000017ECF5CD700>, <__main__.Edge object at 0x0000017ECF5CD490>, <__main__.Edge object at 0x0000017ECF5CD040>]


In [66]:
alpha = drain[0].alpha
const = drain[0].const
p0 = drain[0].p_0
# Нахождение уравнений от-но дебитов каждой скважины
def find_Q_well(p0):
    p0 = p0
    Q_well = []
    for well in range(len(wells)):
        Q_well.append(alpha * (p0 + sum(edge.pressure_loss for edge in pipes_from_wells_to_drain[well])) + const)
    return Q_well

flow_rate = []
flow_rate = symbols(['Q{}'.format(i+1) for i in range(len(Q_well))])
print(flow_rate)

[Q1, Q2]


In [77]:
# while True:
p0 = drain[0].p_0
Q_well = find_Q_well(p0)
flow_rate = symbols(['Q{}'.format(i+1) for i in range(len(Q_well))])

# solve the system of equations
eqs = [Eq(Q_well[i], flow_rate[i]) for i in range(len(Q_well))]
sol = solve(eqs, flow_rate)

# calculate the new value of Q0
Q0 = sum(sol.get(well) for well in flow_rate)

# check for convergence
# if abs(Q0 - Q0_prev) < eps:
#     break

# update the previous value of Q0
Q0_prev = Q0

# update the value of p0
p0 = (Q0-const)/alpha

# print the final values of Q0 and p0
print("Q0:", Q0)
print("p0:", p0)

Q0: -0.206735578526175
p0: -3.40224519284206


In [76]:
# # Итерационный алгоритм
# Q0_0 = alpha * drain[0].p_0 + const
# eps = 0.001

# p0 = drain[0].p_0
# Q_well = find_Q_well(p0)
# print(Q_well)
# # Решение СЛАУ
# eqs = []
# for i in range(len(Q_well)):
#     eqs.append(Eq(Q_well[i], flow_rate[i]))
# sol = solve(eqs, flow_rate)
# drain
# print(sol)
# print(sol.get(flow_rate[0]))
# Q0 = sum(sol.get(well) for well in flow_rate)
# print(Q0)
# p0 = (Q0-const)/alpha

[2250.0*Q1 + 750.0*Q2 + 310, 750.0*Q1 + 2250.0*Q2 + 310]
{Q1: -0.103367789263088, Q2: -0.103367789263088}
-0.103367789263088
-0.206735578526175


## Тестовая часть, понадлежит удалению перед отправкой

In [7]:
from sympy import symbols, Eq, solve

class Well:
    def __init__(self, idx, alpha, const, Q):
        self.idx = idx
        self.alpha = alpha
        self.const = const
        self.Q = Q

          
class Vertex:
    def __init__(self, idx):
        self.idx = idx
        self.edge_input = []
        self.Q = 0

    def update_Q(self):
        self.Q = sum(edge.Q_edge for edge in self.edge_input)

class Edge:
    def __init__(self, idx, vertex1, vertex2, length = 5, diameter = 2, roughness = 0.1, density = 1000):
        self.idx = idx
        self.vertex1 = vertex1
        self.vertex2 = vertex2
        self.length = length
        self.diameter = diameter
        self.roughness = roughness
        self.density = density
        self.update_Q_edge()

    def update_Q_edge(self):
        self.Q_edge = self.vertex1.Q
        self.pressure_loss = self.density * self.length * self.Q_edge * self.roughness / self.diameter

class Drain:
    def __init__(self, p_0, alpha = 2, const = 10, idx=0):
        self.p_0 = p_0
        self.edge_input = []
        self.alpha = alpha
        self.const = const
        self.idx = idx
        self.Q_0 = self.alpha * self.p_0 + self.const

# Body of the algorithm
def create_graph():
    # get the number of Wells
    num_wells = int(input("Enter the number of Wells: "))
    wells = []
    
    # Создание списка из num_wells элементов - Q1, Q2, Q3
    flow_rate = []
    for i in range(num_wells):
        flow_rate.append('Q' + str(i+1))
    flow_rate = symbols(flow_rate)
    
    counter = 0
    for i in range(num_wells):
        counter = counter + 1
        wells.append(Well(i+1, alpha, const, flow_rate[i]))

    # get the number of Vertex
    num_vertex = int(input("Enter the number of Vertex: "))
    vertex = []
    for i in range(num_vertex):
        vertex.append(Vertex(i + counter))

    # Create Drain
    drain = [Drain(100)]
    common_vertex = drain + wells + vertex # A common array of all vertices
    print(common_vertex)

    # get the number of Edges
    num_edges = int(input("Enter the number of Edges: "))
    edges = []
    for i in range(num_edges):
        vertex1 = int(input("Enter the first vertex for edge {}: ".format(i + 1)))
        vertex2 = int(input("Enter the second vertex for edge {}: ".format(i + 1)))
        edges.append(Edge(i+1, common_vertex[vertex1], common_vertex[vertex2]))

    for edge in edges:
        edge.vertex2.edge_input.append(edge)

    while edges[num_edges-1].Q_edge == 0:
        
        for edge in edges:
            edge.update_Q_edge()

        for vert in vertex:
            vert.update_Q()

    return wells, vertex, drain, edges

wells, vertex, drain, edges = create_graph()

5


In [1]:
from sympy import symbols, Eq, solve


variables = ['x', 'y', 'z']
variables = symbols(variables)
print(variables)
print(type(variables[0]))

variables = ['x', 'y', 'z']
x, y, z = symbols(','.join(variables))
print(x, y, z)
print(type(x))


[x, y, z]
<class 'sympy.core.symbol.Symbol'>
x y z
<class 'sympy.core.symbol.Symbol'>


In [ ]:
input().strip().split()
print(a)

In [5]:
b = []
for i in range(3):
        b.append('Q' + str(i))

print(b)

['Q0', 'Q1', 'Q2']


In [2]:
i = 1
vertex1 = int(input("Enter the first vertex for edge {}: ".format(i + 1)))
print(vertex1)

1


In [21]:
drain = [Drain(100)]
print(drain[0].idx)

0


In [26]:
a = []
a.append(1)
print(a)

[1]
